# Momentum trading strategy
As from Quantitative Trading by Ernest Chan, there are the paradigms of mean reversion vs momentum. I believe that crypto assets are much more momentum based given their speculative nature, and the sheer number of retail traders. 

I will be implementing that hypothesis in this notebook.

In [26]:
from dotenv import load_dotenv
import os
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import mplfinance as mpf

In [27]:
load_dotenv()

False

# Data loading

In [28]:
BTCUSDT_FOLDER_PATH = os.getenv("BTCUSDT_FOLDER_PATH")

In [29]:
def retrieve_csv_files(directory):
    csv_files = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            csv_files.append(os.path.join(directory, file))
    csv_files.sort()

    return csv_files

In [50]:
btcusdt_csv_files = retrieve_csv_files(BTCUSDT_FOLDER_PATH)
btcusdt_csv_files

[]

In [31]:
columns = [
    'open time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close time',
    'quote asset volume',
    'number of trades',
    'taker buy base asset volume',
    'taker buy quote asset volume',
    'ignore'
]

In [32]:
# have to process these data differently as units of time
# changed at the start of 2025
df_before_2025 = pd.DataFrame(columns=columns)
df_2025 = pd.DataFrame(columns=columns)

In [33]:
for file in btcusdt_csv_files:
    filename = os.path.basename(file)
    temp_df = pd.read_csv(file, names=columns)
    if '2024' in filename:
        df_before_2025 = pd.concat([df_before_2025, temp_df])
    else:
        df_2025 = pd.concat([df_2025, temp_df])

In [34]:
df_before_2025.shape, df_2025.shape

((0, 12), (0, 12))

In [35]:
df_before_2025

,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume,ignore


In [36]:
df_2025

,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume,ignore


In [37]:
df_before_2025['open_timestamp'] = pd.to_datetime(df_before_2025['open time'], unit='ms')
df_before_2025.set_index('open_timestamp', inplace=True)

df_2025['open_timestamp'] = pd.to_datetime(df_2025['open time'], unit='us')
df_2025.set_index('open_timestamp', inplace=True)

In [38]:
df = pd.concat([df_before_2025, df_2025])
df

,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume,ignore
open_timestamp,,,,,,,,,,,,


## Data quality check
### Check for missing data

In [39]:
time_diff = df.reset_index()['open_timestamp'].diff()

expected_interval = pd.Timedelta('1 hour')
missing_data = time_diff != expected_interval

if missing_data.any():
    print('Missing data')
    print(time_diff[missing_data])

In [40]:
print(df.iloc[2448:2467]) 

Empty DataFrame
Columns: [open time, open, high, low, close, volume, close time, quote asset volume, number of trades, taker buy base asset volume, taker buy quote asset volume, ignore]
Index: []


There are 7 hours of missing data on 2024-11-19. We will create a new column called `data_quality` where we flag it to be `pass` or `fail`. We will run our backtest on only those data points where the data quality is `pass`.

In [41]:
df['data_quality'] = 'pass'

gap_start_index = 2448
gap_end_index = 2466

df.loc[df.index[gap_start_index:gap_end_index], 'data_quality'] = 'fail'

In [42]:
df.iloc[2448:2467]

,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume,ignore,data_quality
open_timestamp,,,,,,,,,,,,,


# Momentum trading strategy
We will implement momentum trading strategy with lookback and relative strength index. I chose momentum over mean reversion as crypto prices in general follow a very strong 'herd mentality'. Let's test out this hypothesis.

In [43]:
def calculate_returns(prices):
    return prices.pct_change()

def calculate_sma(prices, lookback_period):
    return prices.rolling(window=lookback_period).mean()

def calculate_rsi(prices, periods=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [44]:
def generate_signals(df, sma, rsi):
    """
    Generate trading signals based on SMA and RSI
    
    Returns:
    Series with values: 1 (buy), -1 (sell), 0 (no position)
    """
    signals = pd.Series(0, index=df.index)
    
    # Basic momentum signals
    signals[df['close'] > sma] = 1  # Buy signal
    signals[df['close'] < sma] = -1  # Sell signal
    
    # Filter signals using RSI
    signals[(signals == 1) & (rsi < 30)] = 0  # Remove oversold signals
    signals[(signals == -1) & (rsi > 70)] = 0  # Remove overbought signals
    
    return signals

In [45]:
def calculate_strategy_returns(price_returns, signals):
    return price_returns * signals.shift(1)

def calculate_cumulative_returns(returns):
    return (1 + returns).cumprod()

In [46]:
def calculate_metrics(strategy_returns, signals):
    # Count actual trades when signal changes
    signal_changes = signals[signals != signals.shift(1)]
    total_trades = len(signal_changes) - 1  # Subtract 1 to exclude the first signal
    
    # Only count returns when we actually have trades
    trade_returns = strategy_returns[signals != 0]  # Only consider returns when we have a position
    
    winning_trades = len(trade_returns[trade_returns > 0])
    losing_trades = len(trade_returns[trade_returns < 0])
    
    # Win rate should be winning_trades / (winning_trades + losing_trades)
    win_rate = winning_trades / (winning_trades + losing_trades) if (winning_trades + losing_trades) > 0 else 0
    
    returns_std = strategy_returns.std()
    sharpe_ratio = (np.sqrt(365 * 24) * strategy_returns.mean() / 
                   returns_std if returns_std != 0 else 0)
    
    return {
        'Total Trades': total_trades,
        'Win Rate': win_rate,
        'Sharpe Ratio': sharpe_ratio,
        'Final Return': calculate_cumulative_returns(strategy_returns).iloc[-1] - 1
    }

In [47]:

def implement_momentum_strategy(df, lookback_period=24, rsi_period=14):
    """
    Implement momentum trading strategy
    
    Parameters:
    df: DataFrame with OHLC data
    lookback_period: Period for SMA calculation (default: 24 hours)
    rsi_period: Period for RSI calculation (default: 14 hours)
    """
    # Filter for valid data
    df = df[df['data_quality'] == 'pass'].copy()
    
    # Calculate indicators
    df['returns'] = calculate_returns(df['close'])
    df['SMA'] = calculate_sma(df['close'], lookback_period)
    df['RSI'] = calculate_rsi(df['close'], rsi_period)
    
    # Generate signals
    df['signal'] = generate_signals(df, df['SMA'], df['RSI'])
    
    # Calculate returns
    df['strategy_returns'] = calculate_strategy_returns(df['returns'], df['signal'])
    
    # Calculate cumulative returns
    df['cumulative_returns'] = calculate_cumulative_returns(df['returns'])
    df['strategy_cumulative_returns'] = calculate_cumulative_returns(df['strategy_returns'])
    
    # Calculate metrics
    metrics = calculate_metrics(df['strategy_returns'], df['signal'])
    
    return df, metrics

In [48]:
def run_strategy(df, lookback_period=24, rsi_period=14):
    """
    Run the momentum strategy and print results
    """
    df, metrics = implement_momentum_strategy(df, lookback_period, rsi_period)
    
    print("\nStrategy Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")
    
    return df, metrics

In [49]:
run_strategy(df)

IndexError: single positional indexer is out-of-bounds

In [ ]:
def analyze_different_periods(df, lookback_periods=[6, 12, 24], rsi_periods=[14, 21, 28]):
    """
    Analyze strategy performance with different combinations of periods
    """
    results = []
    
    for lookback in lookback_periods:
        for rsi in rsi_periods:
            _, metrics = implement_momentum_strategy(df, lookback, rsi)
            results.append({
                'Lookback Period': lookback,
                'RSI Period': rsi,
                'Final Return': metrics['Final Return'],
                'Sharpe Ratio': metrics['Sharpe Ratio'],
                'Win Rate': metrics['Win Rate']
            })
    
    return pd.DataFrame(results)

In [ ]:
def find_optimal_periods(df):
    """
    Test different period combinations and find the best performing ones
    """
    # Test shorter lookback periods
    lookback_periods = [4, 6, 8, 12]
    rsi_periods = [7, 14, 21]
    
    results_df = analyze_different_periods(df, lookback_periods, rsi_periods)
    
    # Sort by Sharpe Ratio (or Final Return, depending on your preference)
    results_df = results_df.sort_values('Sharpe Ratio', ascending=False)
    
    print("\nTop 5 Period Combinations:")
    print(results_df.head())
    
    return results_df

In [ ]:
experiment_df = find_optimal_periods(df)
experiment_df


Top 5 Period Combinations:
    Lookback Period  RSI Period  Final Return  Sharpe Ratio  Win Rate
7                 8          14     -0.436039     -2.005519  0.475402
8                 8          21     -0.440276     -2.037773  0.475047
6                 8           7     -0.482860     -2.338488  0.469940
11               12          21     -0.507456     -2.530223  0.469947
10               12          14     -0.508330     -2.532399  0.469133


,Lookback Period,RSI Period,Final Return,Sharpe Ratio,Win Rate
7,8,14,-0.436039,-2.005519,0.475402
8,8,21,-0.440276,-2.037773,0.475047
6,8,7,-0.482860,-2.338488,0.469940
11,12,21,-0.507456,-2.530223,0.469947
10,12,14,-0.508330,-2.532399,0.469133
2,4,21,-0.504677,-2.534291,0.475078
9,12,7,-0.524059,-2.657201,0.466651
1,4,14,-0.539683,-2.852014,0.476356
0,4,7,-0.559883,-3.081548,0.481606
4,6,14,-0.602649,-3.414575,0.471948
